In [9]:
##Importing Modules
import M3_Image_Class
from importlib import reload
reload(M3_Image_Class)
from M3_Image_Class import M3_Stamp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tf
import rasterio as rio
import os
from os import path as path
import time

In [3]:
##Loading in smoothed spectral images
ice_pipe_folder = 'D:/Data/Ice_Pipeline_Out_7-19-23'
smooth_folder = path.join(ice_pipe_folder,'rfl_smooth')

def find(s,c):
    return [n for n,i in enumerate(s) if i==c]

in_dict = {}
prog,tot = 1,len(os.listdir(smooth_folder))
for file in os.listdir(smooth_folder):
    im = tf.imread(path.join(smooth_folder,file))
    in_dict.update({file[:find(file,'_')[0]]:im})
    print (f'\r{prog} of {tot} ({prog/tot:.1%})',end='\r')
    prog+=1

In [20]:
##Running destriping step
folder_path = 'D:/Data/Ice_Pipeline_Out_8-7-23'
print (f'Analyzing: {folder_path}')
all_rfl_paths = [os.path.join(folder_path,'rfl_cropped',i) for i in os.listdir(os.path.join(folder_path,'rfl_cropped'))]
all_loc_paths = [os.path.join(folder_path,'loc_cropped',i) for i in os.listdir(os.path.join(folder_path,'loc_cropped'))]
all_obs_paths = [os.path.join(folder_path,'obs_cropped',i) for i in os.listdir(os.path.join(folder_path,'obs_cropped'))]

with open(os.path.join(folder_path,'stampNames.txt')) as f:
    stamp_names = f.readlines()
stamp_names = [i[:-2] for i in stamp_names]

prog,tot = 0,len(stamp_names)
start_time = time.time()
for rfl_path,loc_path,obs_path,stamp_name in zip(all_rfl_paths,all_loc_paths,all_obs_paths,stamp_names):
    rfl_im,loc_im,obs_im = tf.imread(rfl_path),tf.imread(loc_path),tf.imread(obs_path)
    stamp_object = M3_Stamp(rfl_im,loc_im,obs_im,stamp_name,folder_path)
    stamp_object.destripe_images(save_step=True)
    prog+=1
    print (f'\rAnalysis for {stamp_name} complete ({prog/tot:.2%})',end='\r')
    del rfl_im,loc_im,obs_im

print (f'All images destriped in {(time.time()-start_time)/60:.2f} minutes')

Select Analysis Folder:
All images destriped in 23.51 minutes (100.00%)


In [ ]:
##Running shadow correction step
from run_correction_step import correction
correction(False)

In [1]:
##Running spectrum smoothing step
from run_spectrum_smoothing_step import spectrum_smoothing
spectrum_smoothing(False)

Select Analysis Folder:
D:/Data/Ice_Pipeline_Out_8-7-23 selected for analysis
Select input folder for spectrum smoothing step:
The rfl_correction folder has been selected as the processing input
Applying spline along axis 2...


KeyboardInterrupt: 

In [ ]:
##Running band position mapping step
from run_band_position_mapping import band_position_mapping
band_position_mapping(False)

In [ ]:
##Running spectral angle mapping step
from run_spectral_angle_mapping import sa_mapping
sa_mapping(False)

In [ ]:
##Running band depth mapping step
from run_band_depth_mapping import bd_mapping
bd_mapping(False)

In [25]:
def get_summary_dataframe(boolean_array:np.ndarray,map_coord_array:np.ndarray,value_name_list:list,*args)->pd.DataFrame:
    true_coords = np.where(boolean_array==True)
    value_array = np.array([i[true_coords] for i in args]).T #N by len(args) array of values at each True boolean pixel
    map_coords = map_coord_array[true_coords[0],true_coords[1],:]

    summary_array = np.concatenate([map_coords,np.array([true_coords[0],true_coords[1]]).T,value_array],axis=1)
    summary_df = pd.DataFrame(summary_array)
    summary_df.columns = ['Longitude','Latitude','Elevation','x','y',*value_name_list]

    return summary_df

In [28]:
##Getting ice water detection maps
criteria = {
    'Band Positioning':True,
    'Spectral Angle':30.0,
    'Band Depth':0.1
}

folder_path = 'D:/Data/Ice_Pipeline_Out_8-7-23'

with open(os.path.join(folder_path,'stampNames.txt')) as f:
    stamp_names = f.readlines()
stamp_names = [i[:-2] for i in stamp_names]

def get_bool_arrays(stamp_name,**kwargs)->tuple[np.ndarray,np.ndarray,np.ndarray]:
    if kwargs.get('Band Positioning')==True:
        array1 = tf.imread(os.path.join(folder_path,'ice_band_bool',f'{stamp_name}.tif'))
    else:
        array1 = np.ones(tf.imread(os.path.join(folder_path,'ice_band_bool',f'{stamp_name}.tif')).shape)
    
    if kwargs.get('Spectral Angle') != False:
        array2 = tf.imread(os.path.join(folder_path,f'spectral_angle_bool_{kwargs.get("Spectral Angle")}',f'{stamp_name}.tif'))
    else:
        array2 = np.ones(array1.shape)

    if kwargs.get('Band Depth') != False:
        array3 = tf.imread(os.path.join(folder_path,f'band_depth_bool_{kwargs.get("Band Depth")}',f'{stamp_name}.tif'))
    else:
        array3 = np.ones(array1.shape)

    return array1,array2,array3


array_list = [get_bool_arrays(i,**criteria) for i in stamp_names]

try:
    os.mkdir(os.path.join(folder_path,'ice_detection_results'))
except:
    pass

new_bool_array = [i*j*k for i,j,k in array_list]
loc_arrays = [tf.imread(os.path.join(folder_path,'loc_cropped',i)) for i in os.listdir(os.path.join(folder_path,'loc_cropped'))]
for bool_array,loc_array,bd_array,sa_array,stamp_name in zip(new_bool_array,loc_arrays,array3,array2,stamp_names):
    print (len(np.where(i)[0]))
    tf.imwrite(os.path.join(folder_path,'ice_detection_results',f'{stamp_name}.tif'),i.astype('float32'))
    summary_df = get_summary_dataframe(bool_array,loc_array,['Band Dep'])

2469


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 2 has 1 dimension(s)